# EasyMoney preprocessing (Purchase Propensity Model)

## 1. Imports

### 1.1. Libraries

In [30]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import sklearn
from sklearn import set_config
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, MinMaxScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, RepeatedKFold, KFold
from sklearn.metrics import accuracy_score
import category_encoders as ce

import folium
import plotly.express as px
from scipy import stats
from sklearn.tree import export_graphviz
import graphviz

import statsmodels.api as sm

set_config(transform_output = "pandas")

In [31]:
print("Working with these versions of libraries\n")
print(f"Numpy version {np.__version__}")
print(f"Pandas version {pd.__version__}")
print(f"Statsmodels version {sm.__version__}")
print(f"Sklearn version {sklearn.__version__}")

Working with these versions of libraries

Numpy version 1.26.4
Pandas version 2.1.4
Statsmodels version 0.14.0
Sklearn version 1.3.0


### 1.2. Data

In [32]:
PATH = r'C:\Users\Usuario\Desktop\Proyects\Easy Money\data_compressed'
file_name =  r"\customer_commercial_activity.csv"
file_name1 = r"\customer_products.csv"
file_name2 = r"\customer_sociodemographics.csv"
file_name3 = r"\product_description.csv"
file_name4 = r"\sales.csv"
cca = pd.read_csv(PATH + file_name, sep = ",", index_col=0)
cp = pd.read_csv(PATH + file_name1, sep = ",", index_col=0)
cs = pd.read_csv(PATH + file_name2, sep = ",", index_col=0)
prd = pd.read_csv(PATH + file_name3, sep = ",", index_col=0)
sales = pd.read_csv(PATH + file_name4, sep = ",", index_col=0)

dfs = [cca, cp, cs, sales]

### 1.3. Custom functions

In [33]:
## Sampling data

def sampling_xdf(dfs, p_sample):
    sampled_dfs = []
    for df in dfs:
        n_rows = int(len(df)*p_sample)
        sampled_df = df.sample(n = n_rows, random_state = 42)
        sampled_dfs.append(sampled_df)
    return sampled_dfs

## Preprocessing

def setOthers(dataframe, column, num_values, fillvalue):
    top_categories = dataframe[column].value_counts().head(num_values)
    top_categories_list = top_categories.index.to_list()
    top_categories_list.append(fillvalue)
    dataframe[column] = pd.Categorical(dataframe[column], categories=top_categories_list)
    return dataframe[column].fillna(fillvalue)

def seniority(df, date_column, threshold_date):
    df[date_column] = pd.to_datetime(df[date_column])

    df['customer_seniority'] = 'Old'
    df.loc[df.groupby('pk_cid')[date_column].transform('min') >= threshold_date, 'customer_seniority'] = 'New'

    return df

## Calculation functions

def calc_moda(series):
    return series.mode().iloc[0]

### 1.4. Sampling data

In [34]:
cca, cp, cs, sales = sampling_xdf(dfs, 0.1)

## 2. Preprocessing

### 2.1. Sales_products

In [35]:
def get_sales_data(sales, prd):
    prd = prd.rename(columns={'pk_product_ID': 'product_ID'})
    sales = sales.rename(columns={'cid' : 'pk_cid'})
    sales['month_sale_int'] = pd.to_datetime(sales['month_sale']).dt.strftime('%Y%m%d').astype(int)
    sales_prd = sales.merge(prd, on= 'product_ID', how= 'inner')

    sales_prd_merge = sales_prd.groupby('pk_cid').agg(
    T_sales = ('pk_sale', 'count'),
    n_product = ('product_desc', 'count'),
    T_net_margin = ('net_margin', 'sum'),
    Mean_net_margin = ('net_margin', 'mean'),
    first_sale = ('month_sale_int', 'min'),
    last_sale = ('month_sale_int', 'max'),
    product_moda = ('product_ID', calc_moda))
    
    return sales_prd_merge

In [36]:
sales_prd_merge = get_sales_data(sales, prd)

In [37]:
sales['cid'].nunique()

22629

In [38]:
sales_prd_merge.duplicated().sum()

8568

### 2.2. Customers information

In [39]:
def cust_top_features(df):
    customers_merge = df.groupby('pk_cid').agg(
        entry_channel_nunique = ('entry_channel_so', pd.Series.nunique),
        entry_channel_most_freq = ('entry_channel_so', calc_moda),
        act_cust_most_freq = ('active_customer', 'mean'),
        act_cust_std = ('active_customer', 'std'),
        afiliation_time = ('afiliation_time', 'max'),
        salary = ('salary_imp', 'mean'),
        age = ('age', 'max'),
        region_code = ('region_code_so', calc_moda),
        entry_date = ('entry_date', 'min'),
        segment = ('segment_so', calc_moda),
        customer_seniority = ('customer_seniority', calc_moda)
        )
    customers_merge['act_cust_std'] = customers_merge['act_cust_std'].fillna(-1)
    return customers_merge

In [40]:
def get_customers_data(cca, cs):
        cust = cca.merge(cs, on = ['pk_cid', 'pk_partition'], how = 'inner')

        cust = cust[cust['deceased']== 'N'].drop('deceased', axis =  1)
        cust_es = pd.DataFrame(cust[cust['country_id'] == 'ES'].drop('country_id', axis = 1))

        cust_es['active_customer'] = cust_es['active_customer'].astype(int)
        cust_es['pk_partition'] = pd.to_datetime(cust_es['pk_partition'])
        cust_es['entry_date'] = pd.to_datetime(cust_es['entry_date'])
        cust_es['afiliation_time'] = cust_es['pk_partition'] - cust_es['entry_date']

        cust_es['gender_mf'] = SimpleImputer(strategy =  'most_frequent').fit_transform(
                cust_es[['gender']])
        sal_mn_regage = cust_es.groupby(['age', 'region_code'])['salary'].transform('mean')
        cust_es['salary_imp'] = cust_es['salary'].fillna(sal_mn_regage).round(2)
        cust_es['salary_imp'] = SimpleImputer(strategy = 'mean').fit_transform(cust_es[['salary_imp']])

        cust_es['segment_so'] = setOthers(cust_es, 'segment', 2, '03 - Other')
        cust_es['entry_channel_so'] = setOthers(cust_es, 'entry_channel', 10, 'ZZZ')
        cust_es['region_code_so'] = setOthers(cust_es, 'region_code', 45, 99.0)
        threshold_date = pd.to_datetime('2018-01-01')
        cust_es = seniority(cust_es, 'entry_date', threshold_date)
        return cust_top_features(cust_es)

In [41]:
cust_es_merge = get_customers_data(cca, cs)

In [42]:
cust_es_merge.duplicated().sum()

19867

### 2.3. Customers products


In [43]:
cp['pk_partition']

6059396     2018-09
10140464    2019-02
8908827     2018-12
7690470     2018-11
13548914    2019-05
             ...   
9741584     2019-01
10962229    2019-03
10081762    2019-02
10601550    2019-02
10001199    2019-02
Name: pk_partition, Length: 596292, dtype: object

In [46]:
def getproductdata(dataset):
  dataset.fillna(0, inplace = True)
  #dataset['pk_partition'] = pd.to_datetime(dataset['pk_partition']).dt.strftime('%Y%m%d').astype(int)
  products = dataset.drop(['pk_cid', 'pk_partition'], axis=1).columns
  dataset['Products_holding_sum'] = dataset[products].sum(axis = 1)
  dataset['Products_contracted'] = dataset.select_dtypes(include = np.number).sum(axis = 1)
  cust_products_merge = dataset.groupby('pk_cid').agg(
    mean_products_contracted = ('Products_contracted','mean'),
    std_products_contracted = ('Products_contracted','std'),
    max_products_contracted = ('Products_contracted','max'),
    min_products_contracted = ('Products_contracted','min'))
  cust_products_merge['std_products_contracted'].fillna(-1, inplace = True)

  return cust_products_merge

In [47]:
cust_products_merge =getproductdata(cp)
cust_products_merge.head()

,mean_products_contracted,std_products_contracted,max_products_contracted,min_products_contracted
pk_cid,,,,
16203,48616.0,-1.000000,48616.0,48616.0
16502,49516.5,4.949747,49520.0,49513.0
17457,52388.5,4.949747,52392.0,52385.0
17590,52770.0,0.000000,52770.0,52770.0
17799,53404.0,-1.000000,53404.0,53404.0


### 2.4. Building dataset

In [86]:
def mergedataset_(cust_es_merge, cust_products_merge, sales_prd_merge):
  full_df = pd.merge(cust_es_merge, cust_products_merge, how = 'inner', left_index = True, right_index = True)
  full_df = pd.merge(full_df, sales_prd_merge, how = 'left', left_index = True, right_index = True)

  full_df['afiliation_days'] = full_df['afiliation_time'].dt.days
  full_df.drop(columns = 'afiliation_time', axis = 1, inplace = True)
  full_df['act_cust_most_freq'] = full_df['act_cust_most_freq'].astype(float)
  full_df.dropna(inplace = True)
  full_df['entry_date_int'] = pd.to_datetime(full_df['entry_date']).dt.strftime('%Y%m%d').astype(int)
  full_df.drop('entry_date', axis= 1, inplace = True)

  return full_df

In [ ]:
def mergedataset(cust_es_merge, cust_products_merge, sales_prd_merge):
  full_df = pd.merge(cust_es_merge, cust_products_merge, how = 'inner', left_index = True, right_index = True)
  full_df = pd.merge(full_df, sales_prd_merge, how = 'left', left_index = True, right_index = True)

  full_df['afiliation_days'] = full_df['afiliation_time'].dt.days
  full_df.drop(columns = 'afiliation_time', axis = 1, inplace = True)
  full_df['act_cust_most_freq'] = full_df['act_cust_most_freq'].astype(float)
  full_df.fillna(0, inplace = True)
  full_df['entry_date_int'] = pd.to_datetime(full_df['entry_date']).dt.strftime('%Y%m%d').astype(int)
  full_df.drop('entry_date', axis= 1, inplace = True)

  return full_df

In [87]:
full_df_ = mergedataset_(cust_es_merge, cust_products_merge, sales_prd_merge)

In [71]:
full_df = mergedataset(cust_es_merge, cust_products_merge, sales_prd_merge)

In [88]:
full_df_.shape

(14861, 22)

In [72]:
full_df.shape

(324676, 22)

In [79]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 324676 entries, 16203 to 1553680
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   entry_channel_nunique     324676 non-null  int64  
 1   entry_channel_most_freq   324676 non-null  object 
 2   act_cust_most_freq        324676 non-null  float64
 3   act_cust_std              324676 non-null  float64
 4   salary                    324676 non-null  float64
 5   age                       324676 non-null  int64  
 6   region_code               324676 non-null  float64
 7   segment                   324676 non-null  object 
 8   customer_seniority        324676 non-null  object 
 9   mean_products_contracted  324676 non-null  float64
 10  std_products_contracted   324676 non-null  float64
 11  max_products_contracted   324676 non-null  float64
 12  min_products_contracted   324676 non-null  float64
 13  T_sales                   324676 non-null  f

In [80]:
def prep_transform(df):
  #numerical_columns_v2 = df.select_dtypes(exclude = object).drop(['region_code', 'product_moda'], axis=1).columns.to_list()
  numerical_columns_v2 = df.select_dtypes(exclude = object).drop('region_code', axis=1).columns.to_list()
  transform_pipe = ColumnTransformer(transformers = [
    ("scaler", MinMaxScaler(), numerical_columns_v2),
    ("encoder", OneHotEncoder(sparse_output = False), ['entry_channel_most_freq', 'segment', 'product_moda']),
    #("encoder", OneHotEncoder(sparse_output = False), ['entry_channel_most_freq', 'segment', 'product_moda']),
    ('ordinal', OrdinalEncoder(), ['region_code', 'customer_seniority'])
    ])
  full_df_trans = transform_pipe.fit_transform(df)
  return full_df_trans

In [81]:
full_df_trans = prep_transform(full_df)

In [89]:
full_df_trans_ = prep_transform(full_df_)

In [82]:
full_df_trans.info()

<class 'pandas.core.frame.DataFrame'>
Index: 324676 entries, 16203 to 1553680
Data columns (total 47 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   scaler__entry_channel_nunique         324676 non-null  float64
 1   scaler__act_cust_most_freq            324676 non-null  float64
 2   scaler__act_cust_std                  324676 non-null  float64
 3   scaler__salary                        324676 non-null  float64
 4   scaler__age                           324676 non-null  float64
 5   scaler__mean_products_contracted      324676 non-null  float64
 6   scaler__std_products_contracted       324676 non-null  float64
 7   scaler__max_products_contracted       324676 non-null  float64
 8   scaler__min_products_contracted       324676 non-null  float64
 9   scaler__T_sales                       324676 non-null  float64
 10  scaler__n_product                     324676 non-null  float64
 11  

In [84]:

em_propensity = full_df_trans.drop_duplicates()
prd_prop_to_merge = cp.drop('pk_partition', axis=1).drop_duplicates().set_index('pk_cid')
em_propensity_pp = pd.merge(em_propensity, prd_prop_to_merge[['Products_contracted','Products_holding_sum', 'pension_plan']], 
                            how = 'left', left_index = True, right_index = True).drop_duplicates()
em_propensity_emc = pd.merge(em_propensity, prd_prop_to_merge[['Products_contracted','Products_holding_sum', 'emc_account']], 
                            how = 'left', left_index = True, right_index = True).drop_duplicates()
em_propensity_ltd = pd.merge(em_propensity, prd_prop_to_merge[['Products_contracted','Products_holding_sum', 'long_term_deposit']], 
                            how = 'left', left_index = True, right_index = True).drop_duplicates()
em_propensity_dc = pd.merge(em_propensity, prd_prop_to_merge[['Products_contracted','Products_holding_sum', 'debit_card']], 
                            how = 'left', left_index = True, right_index = True).drop_duplicates()

In [91]:
em_propensity_ = full_df_trans_.drop_duplicates()
prd_prop_to_merge = cp.drop('pk_partition', axis=1).drop_duplicates().set_index('pk_cid')
em_propensity_pp_ = pd.merge(em_propensity_, prd_prop_to_merge[['Products_contracted','Products_holding_sum', 'pension_plan']], 
                            how = 'left', left_index = True, right_index = True).drop_duplicates()
em_propensity_emc_ = pd.merge(em_propensity_,prd_prop_to_merge[['Products_contracted','Products_holding_sum', 'emc_account']], 
                            how = 'left', left_index = True, right_index = True).drop_duplicates()
em_propensity_ltd_ = pd.merge(em_propensity_, prd_prop_to_merge[['Products_contracted','Products_holding_sum', 'long_term_deposit']], 
                            how = 'left', left_index = True, right_index = True).drop_duplicates()
em_propensity_dc_ = pd.merge(em_propensity_, prd_prop_to_merge[['Products_contracted','Products_holding_sum', 'debit_card']], 
                            how = 'left', left_index = True, right_index = True).drop_duplicates()

In [85]:
pd.to_pickle(em_propensity_pp,"C:/Users/Usuario/Desktop/Proyects/Easy Money/EasyMoney_/pickles/em_propensity_pp")
pd.to_pickle(em_propensity_emc,"C:/Users/Usuario/Desktop/Proyects/Easy Money/EasyMoney_/pickles/em_propensity_emc")
pd.to_pickle(em_propensity_ltd,"C:/Users/Usuario/Desktop/Proyects/Easy Money/EasyMoney_/pickles/em_propensity_ltd")
pd.to_pickle(em_propensity_dc,"C:/Users/Usuario/Desktop/Proyects/Easy Money/EasyMoney_/pickles/em_propensity_dc")

In [92]:
pd.to_pickle(em_propensity_pp_,"C:/Users/Usuario/Desktop/Proyects/Easy Money/EasyMoney_/pickles/em_propensity_pp_")
pd.to_pickle(em_propensity_emc_,"C:/Users/Usuario/Desktop/Proyects/Easy Money/EasyMoney_/pickles/em_propensity_emc_")
pd.to_pickle(em_propensity_ltd_,"C:/Users/Usuario/Desktop/Proyects/Easy Money/EasyMoney_/pickles/em_propensity_ltd_")
pd.to_pickle(em_propensity_dc_,"C:/Users/Usuario/Desktop/Proyects/Easy Money/EasyMoney_/pickles/em_propensity_dc_")